# Exploratory Data Analysis on Drive and Act Dataset:
*  Check the Resolution of the videos
*  Check the Frame rate of the videos
*  Total number of Frames
*  Height and Width of the video
*  Pulling in Images from Video


In [2]:
!nvidia-smi

Thu Aug 10 11:50:59 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:06:00.0 Off |                    0 |
| N/A   33C    P0    43W / 300W |      3MiB / 32768MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  Off  | 00000000:07:00.0 Off |                    0 |
| N/A   

In [3]:
import torch

if torch.cuda.is_available():
    device = torch.device('cuda')
    print('GPU device:',torch.cuda.get_device_name(0))
else:
    device = torch.device('cpu')
    print('No GPU avaialable, Using CPU')

torch.cuda.set_device(0)

/home/sur06423/miniconda3/envs/vi_trans/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


GPU device: Tesla V100-SXM2-32GB


In [4]:
pwd

'/home/sur06423/hiwi/vit_exp/vision_tranformer_baseline/notebooks'

In [5]:
input_file_path = '/home/sur06423/hiwi/vit_exp/vision_tranformer_baseline/data/kinect_color/kinect_color/vp1/run1b_2018-05-29-14-02-47.kinect_color.mp4'

# Resolution of the video

* "Resolution: {width}x{height}" : 960x540
* "Frame Rate: {frame_rate} FPS" : 15 FPS

In [6]:
import cv2

# Path to the video file
video_path = "/home/sur06423/hiwi/vit_exp/vision_tranformer_baseline/data/kinect_color/kinect_color/vp1/run1b_2018-05-29-14-02-47.kinect_color.mp4"

# Open the video file
cap = cv2.VideoCapture(video_path)

# Check if the video file was opened successfully
if not cap.isOpened():
    print("Error: Could not open video file")
else:
    # Get video resolution and frame rate
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frame_rate = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    print(f"Resolution: {width}x{height}")
    print(f"Frame Rate: {frame_rate} FPS")
    print(f"Total Frames: {total_frames}")

    # Release the video capture object and close the display window
    cap.release()
    cv2.destroyAllWindows()


Resolution: 960x540
Frame Rate: 15 FPS
Total Frames: 19072


In [9]:
import csv

def check_annotation_errors(annotation_file_path):
    annotations = []
    error_annotations = []

    with open(annotation_file_path, 'r') as csv_file:
        csv_reader = csv.reader(csv_file)
        header = next(csv_reader)  # Skip the header
        
        for row in csv_reader:
            participant_id, file_id, annotation_id, frame_start, frame_end, activity, chunk_id = row
            
            existing_annotations = [
                anno for anno in annotations
                if (
                    anno['annotation_id'] == annotation_id and
                    anno['chunk_id'] == chunk_id and
                    anno['activity'] == activity and
                    anno['participant_id'] == participant_id and
                    anno['file_id'] == file_id
                )
            ]
            
            if existing_annotations:
                error_annotations.extend(existing_annotations)
                error_annotations.append({
                    'participant_id': participant_id,
                    'file_id': file_id,
                    'annotation_id': annotation_id,
                    'frame_start': frame_start,
                    'frame_end': frame_end,
                    'activity': activity,
                    'chunk_id': chunk_id
                })
            else:
                annotations.append({
                    'participant_id': participant_id,
                    'file_id': file_id,
                    'annotation_id': annotation_id,
                    'frame_start': frame_start,
                    'frame_end': frame_end,
                    'activity': activity,
                    'chunk_id': chunk_id,
                })

    if error_annotations:
        error_filename = 'error_annotation_3.csv'
        with open(error_filename, 'w', newline='') as error_file:
            error_writer = csv.writer(error_file)
            error_writer.writerow(header)
            for error_row in error_annotations:
                error_writer.writerow([
                    error_row['participant_id'],
                    error_row['file_id'],
                    error_row['annotation_id'],
                    error_row['frame_start'],
                    error_row['frame_end'],
                    error_row['activity'],
                    error_row['chunk_id']
                ])
        print(f'Error annotations written to {error_filename}')
    else:
        print('No error annotations found.')

# Provide the path to the annotation CSV file
annotation_file_path = '/home/sur06423/hiwi/vit_exp/vision_tranformer_baseline/data/kinect_color_annotation/activities_3s/kinect_color/midlevel.chunks_90.split_0.val.csv'
check_annotation_errors(annotation_file_path)


No error annotations found.
